In [84]:
# Example applying a multi-sensor spectral soil library (XRF, GRS, vis and NIR) for soil fertility attributes quantification (example for exCa)

# instantiating the necessary libraries
import numpy as np
import pandas as pd
import os
pd.options.plotting.backend = 'plotly'

#let's start by importing the data
XGRS = pd.read_csv('datasets/XGRS.csv', header=0, sep=';')
XVIS = pd.read_csv('datasets/XVIS.csv', header=0, sep=';')
XNIR = pd.read_csv('datasets/XNIR.csv', header=0, sep=';')
XXRF = pd.read_csv('datasets/XXRF.csv', header=0, sep=';')
Y = pd.read_csv('datasets/Y.csv', header=0, sep=';')


## **kennard-stone**

In [85]:
# Applying the Kennard-Stone for splittng calibration and prediction sets
import kennard_stone as ks

Ycal, Ypred = ks.train_test_split(Y.drop(['Samples'], axis=1), test_size = 0.30)
indices_cal = Ycal.index
indices_pred = Ypred.index
Ycal.insert(0, 'Samples', Y['Samples'].iloc[indices_cal])
Ycal = Ycal.reset_index(drop=True)
Ypred.insert(0, 'Samples', Y['Samples'].iloc[indices_pred])
Ypred = Ypred.reset_index(drop=True)

Calculating pairwise distances using scikit-learn.
Calculating pairwise distances using scikit-learn.


c:\Users\Usuario\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\Usuario\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\Usuario\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [86]:
# Reproducing the same calibration and prediction splitting for each individual sensor dataset

XcalXRF = XXRF.iloc[indices_cal].reset_index(drop=True)
XpredXRF = XXRF.iloc[indices_pred].reset_index(drop=True)
XcalNIR = XNIR.iloc[indices_cal].reset_index(drop=True)
XpredNIR = XNIR.iloc[indices_pred].reset_index(drop=True)
XcalGRS = XGRS.iloc[indices_cal].reset_index(drop=True)
XpredGRS = XGRS.iloc[indices_pred].reset_index(drop=True)
XcalVIS = XVIS.iloc[indices_cal].reset_index(drop=True)
XpredVIS = XVIS.iloc[indices_pred].reset_index(drop=True)

# **preprocessings**

In [87]:
import preprocessings as prepr # poisson scaling by particular library

XcalXRF_pre, meancal_xrf, meancalpoisson_xrf = prepr.poisson(XcalXRF, mc=True)
XpredXRF_pre = (XpredXRF / np.sqrt(meancal_xrf)) - meancalpoisson_xrf

In [88]:
from scipy.signal import savgol_filter # SAVGOL smoothing

XcalNIR_pre = pd.DataFrame(savgol_filter(XcalNIR,
                                        window_length=11,
                                        polyorder=1,
                                        deriv=1))

XpredNIR_pre = pd.DataFrame(savgol_filter(XpredNIR,
                                        window_length=11,
                                        polyorder=1,
                                        deriv=1))

XcalNIR_pre, meancal_nir = prepr.mc(XcalNIR_pre)
XpredNIR_pre = XpredNIR_pre - meancal_nir

In [89]:
XcalVIS_pre = pd.DataFrame(savgol_filter(XcalVIS,
                                        window_length=3,
                                        polyorder=1,
                                        deriv=1))

XpredVIS_pre = pd.DataFrame(savgol_filter(XpredVIS,
                                        window_length=3,
                                        polyorder=1,
                                        deriv=1))

XcalVIS_pre, meancal_VIS = prepr.mc(XcalVIS_pre)
XpredVIS_pre = XpredVIS_pre - meancal_VIS

In [90]:
XcalGRS_pre = pd.DataFrame(savgol_filter(XcalGRS,
                                        window_length=11,
                                        polyorder=1,
                                        deriv=1))

XpredGRS_pre = pd.DataFrame(savgol_filter(XpredGRS,
                                        window_length=11,
                                        polyorder=1,
                                        deriv=1))

XcalGRS_pre, meancal_grs = prepr.mc(XcalGRS_pre)
XpredGRS_pre = XpredGRS_pre - meancal_grs

# **individual models**

Lets use the individual modeling function to extract the LV scores from the individual spectra, then combine them at mid-level data fusion

In [91]:
import automated_datafusion as df
overview_xrf, calres_xrf, LVscorescal_xrf, predres_xrf, LVscorespred_xrf = df.modelo_individual_otimizado(Xcal=XcalXRF_pre,
                                   ycal=Ycal, 
                                   Xpred=XpredXRF_pre, 
                                   ypred=Ypred,
                                   model='pls', # by using PLS
                                   maxLV=3,
                                   target='exCa',
                                   LVscores=True)                    
overview_xrf

,LVs number,R2 Cal,r2 Cal,RMSEC,R2 CV,r2 CV,RMSECV,Bias CV,tbias CV,RPD CV,RPIQ CV,R2 Pred,r2 Pred,RMSEP,Bias Pred,tbias Pred,RPD Pred,RPIQ Pred
0,1,0.597204,0.597204,1.223043,0.494973,0.495112,1.369483,0.022517,0.121954,1.419892,1.911305,0.570256,0.640980,0.992197,-0.030270,0.146381,1.558249,1.977933
1,2,0.626063,0.626063,1.178416,0.311194,0.435250,1.599365,-0.092499,0.429635,1.215806,1.636587,0.657090,0.755785,0.886305,-0.020249,0.109595,1.744421,2.214248
2,3,0.670572,0.670572,1.106062,0.256313,0.423179,1.661860,-0.139772,0.625962,1.170085,1.575043,0.706116,0.841085,0.820505,-0.127243,0.752841,1.884315,2.391819


In [92]:
overview_nir, calres_nir, LVscorescal_nir, predres_nir, LVscorespred_nir = df.modelo_individual_otimizado(Xcal=XcalNIR_pre,
                                   ycal=Ycal,
                                   target='exCa', 
                                   model='pls', #by using RF
                                   Xpred=XpredNIR_pre, 
                                   ypred=Ypred,
                                   maxLV=5,
                                   LVscores=True)
overview_nir

,LVs number,R2 Cal,r2 Cal,RMSEC,R2 CV,r2 CV,RMSECV,Bias CV,tbias CV,RPD CV,RPIQ CV,R2 Pred,r2 Pred,RMSEP,Bias Pred,tbias Pred,RPD Pred,RPIQ Pred
0,1,0.106437,0.106437,1.821637,-0.119356,0.000039,2.038841,-0.014794,0.053815,0.953737,1.283818,-0.077184,0.015930,1.570862,-0.255133,0.789401,0.984230,1.249314
1,2,0.227962,0.227962,1.693240,-0.128460,0.011305,2.047115,-0.031737,0.114990,0.949882,1.278629,0.004129,0.073399,1.510409,-0.264767,0.853906,1.023623,1.299317
2,3,0.390379,0.390379,1.504628,-0.205615,0.028494,2.115940,0.049654,0.174082,0.918985,1.237039,0.104486,0.172787,1.432285,-0.266749,0.909083,1.079457,1.370189
3,4,0.627317,0.627317,1.176438,-0.165254,0.051931,2.080221,0.033003,0.117675,0.934765,1.258280,0.088146,0.252745,1.445293,-0.502887,1.779922,1.069742,1.357857
4,5,0.689114,0.689114,1.074483,-0.226581,0.022488,2.134260,-0.019573,0.068015,0.911097,1.226421,0.115700,0.306725,1.423288,-0.487797,1.749618,1.086281,1.378850


In [93]:
overview_vis, calres_vis, LVscorescal_vis, predres_vis, LVscorespred_vis = df.modelo_individual_otimizado(Xcal=XcalVIS_pre,
                                   ycal=Ycal,
                                   target='exCa', 
                                   model='pls', #by using RF
                                   Xpred=XpredVIS_pre, 
                                   ypred=Ypred,
                                   maxLV=7,
                                   LVscores=True)
overview_vis

,LVs number,R2 Cal,r2 Cal,RMSEC,R2 CV,r2 CV,RMSECV,Bias CV,tbias CV,RPD CV,RPIQ CV,R2 Pred,r2 Pred,RMSEP,Bias Pred,tbias Pred,RPD Pred,RPIQ Pred
0,1,0.017207,0.017207,1.910426,-0.040232,0.001041,1.965461,0.026940,0.101662,0.989344,1.331749,-0.023421,0.021250,1.531159,-0.148376,0.466934,1.009751,1.281709
1,2,0.035196,0.035196,1.892861,-3.456953,0.001811,4.068349,0.446570,0.819003,0.477962,0.643381,-0.032633,0.000970,1.538035,-0.147776,0.462929,1.005237,1.275979
2,3,0.126685,0.126685,1.800880,-3.402412,0.002694,4.043380,0.516760,0.955656,0.480914,0.647355,0.057233,0.085894,1.469587,-0.199994,0.658787,1.052057,1.335409
3,4,0.166190,0.166190,1.759676,-7.304227,0.003436,5.553270,0.764164,1.030316,0.350157,0.471344,-0.018257,0.047561,1.527291,-0.201249,0.637500,1.012309,1.284955
4,5,0.191685,0.191685,1.732565,-7.722918,0.003160,5.691544,0.755401,0.993088,0.341650,0.459893,-0.058415,0.045949,1.557116,-0.134587,0.416077,0.992919,1.260343
5,6,0.222725,0.222725,1.698973,-2.542763,0.010018,3.627189,0.306625,0.629182,0.536095,0.721633,0.047456,0.085863,1.477188,-0.010368,0.033662,1.046644,1.328538
6,7,0.238541,0.238541,1.681599,-1.964792,0.012464,3.318152,0.150719,0.337210,0.586024,0.788843,0.125381,0.129684,1.415476,-0.019480,0.066008,1.092276,1.386460


In [94]:
overview_grs, calres_grs, LVscorescal_grs, predres_grs, LVscorespred_grs = df.modelo_individual_otimizado(Xcal=XcalGRS_pre,
                                   ycal=Ycal,
                                   target='exCa', 
                                   model='pls', #by using RF
                                   Xpred=XpredGRS_pre, 
                                   ypred=Ypred,
                                   maxLV=3,
                                   LVscores=True)
overview_grs

,LVs number,R2 Cal,r2 Cal,RMSEC,R2 CV,r2 CV,RMSECV,Bias CV,tbias CV,RPD CV,RPIQ CV,R2 Pred,r2 Pred,RMSEP,Bias Pred,tbias Pred,RPD Pred,RPIQ Pred
0,1,0.019264,0.019264,1.908426,-0.013790,0.001014,1.940320,-0.000252,0.000963,1.002163,1.349005,0.025144,0.026664,1.494388,-0.020929,0.067173,1.034598,1.313247
1,2,0.168363,0.168363,1.757382,0.057080,0.080349,1.871270,-0.000979,0.003879,1.039143,1.398782,0.077725,0.134270,1.453528,-0.118199,0.391286,1.063681,1.350163
2,3,0.544487,0.544487,1.300617,0.317041,0.328318,1.592562,-0.034785,0.162022,1.221000,1.643578,0.007046,0.173212,1.508195,0.023937,0.076126,1.025126,1.301224


# **high-level data fusion**

Dictionaries (**cal** and **pred**) whose keys contain the individual predictions of each individual models must be inputed. Then, MLR models will be constructed for all combinations between the keys.

In [95]:
import automated_datafusion as df

predictions_scal_dict = {
    'xrf': calres_xrf,
    'nir': calres_nir,
    'vis': calres_vis,
    'grs': calres_grs
}

predictions_pred_dict = {
    'xrf': predres_xrf,
    'nir': predres_nir,
    'vis': predres_vis,
    'grs': predres_grs
}

results_high_level = df.high_level_fusion_automatizado(predictions_scal_dict, predictions_pred_dict,Ycal,Ypred, target='exCa')

In [96]:
rows = []

for combination in results_high_level.keys():
    row = {
        'Combination': combination,
    }
    row.update(results_mid_level[combination]['metrics'])
    
    rows.append(row)

metrics_high_level = pd.DataFrame(rows)
metrics_high_level.sort_values('RMSEP', axis=0, ascending=True)


,Combination,R2 Cal,r2 Cal,RMSEC,R2 Pred,r2_pred,RMSEP,Bias Pred,tbias Pred,RPD Pred,RPIQ Pred
1,xrf_vis,0.674754,0.674754,1.099019,0.699386,0.821903,0.829847,-0.111369,0.649495,1.823875,2.364894
8,xrf_vis_grs,0.725542,0.725542,1.009571,0.627839,0.657243,0.923333,-0.058959,0.306864,1.639210,2.125451
2,xrf_grs,0.725296,0.725296,1.010024,0.627035,0.655905,0.924330,-0.061955,0.322175,1.637443,2.123160
10,xrf_nir_vis_grs,0.828687,0.828687,0.797617,0.561138,0.604001,1.002668,-0.312293,1.571909,1.509510,1.957278
7,xrf_nir_grs,0.828554,0.828554,0.797927,0.559478,0.601457,1.004562,-0.309326,1.552155,1.506664,1.953588
0,xrf_nir,0.822928,0.822928,0.810913,0.552449,0.607063,1.012545,-0.352834,1.782915,1.494785,1.938186
6,xrf_nir_vis,0.822928,0.822928,0.810912,0.552378,0.606942,1.012625,-0.352684,1.781891,1.494667,1.938032
9,nir_vis_grs,0.746848,0.746848,0.969593,0.238428,0.346329,1.320835,-0.312404,1.167433,1.145894,1.485803
4,nir_grs,0.742842,0.742842,0.977235,0.225556,0.341445,1.331950,-0.329228,1.223384,1.136332,1.473404
3,nir_vis,0.705373,0.705373,1.046009,0.142450,0.311996,1.401595,-0.430165,1.546533,1.079867,1.400190


In [97]:
import plotly.express as px

# Extracting the data for the bar chart
rmseps = metrics_high_level['RMSEP']
combinations = metrics_high_level['Combination']

# Creating the bar chart
fig = px.bar(
    metrics_high_level,
    x=combinations,
    y=rmseps,
    color=combinations,  # Change bar color by the X axis
    title='Comparison of RMSEPs of All Models',
    labels={'x': 'Combination', 'y': 'RMSEP'}
)

# Display the plot
fig.show()